In [4]:
# collect text data using Twitter API
!pip install tweepy

In [5]:
!pip install openpyxl

In [8]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

# Initializing Twitter API

In [10]:
consumer_key = "DreBJqr1OxQZGAuMJIS3kaeTL"
consumer_secret = "5dMdE5ptm468W5o5vQf48AMShNnYGQDvtRkuhcIyycz4wBGKIj"
access_token = "1322035865615556609-z6ZRy7ZJFgNNNleEE4u6zRZ7pNTvTm"
access_token_secret = "Ty0OUxG0ng1YvqihRiDFmhAvOH7j2QhNBbUzWbBi07CH9"

auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

query = "Narendra Modi Prime Minister"

Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

# for tweet in Tweets:
#     print(tweet,"\n")
#     print(".....................................\n")

# Creating Dataframe by fetching data using tweepy

In [11]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [12]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass
stream(data=[query] , file_name='my_tweets')

In [13]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Piyush cahturvedi,6842,102,India,False,0,4,2020-11-22 08:07:44
1,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Надежда Румянцева,151647,1372,,False,0,4,2020-11-22 08:07:14
2,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Shailesh,5637,55,Bharat,False,0,4,2020-11-22 08:06:38
3,RT @SortedEagle: According to news agency ANI'...,🇮🇳 દિવ્યમશુ સરકાર 🇮🇳,305340,5623,India,False,0,11,2020-11-22 08:05:55
4,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Ritesh Salot,121,12,,False,0,4,2020-11-22 08:05:55


In [16]:
!pip install textblob
#Sentiment(polarity, subjectivity) 
#Polarity score: [-1 , +1] 
#Subjectivity: [0.0 , 1.0] where 0.0 is very Objective and 1.0 is very Subjective.

# Cleaning Tweets and analyzing using TextBlob

In [27]:
from textblob import TextBlob

import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))


df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Piyush cahturvedi,6842,102,India,False,0,4,2020-11-22 08:07:44,RT Spoke to MulayamSinghYadav and greeted him ...,Positive
1,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Надежда Румянцева,151647,1372,,False,0,4,2020-11-22 08:07:14,RT Spoke to MulayamSinghYadav and greeted him ...,Positive
2,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Shailesh,5637,55,Bharat,False,0,4,2020-11-22 08:06:38,RT Spoke to MulayamSinghYadav and greeted him ...,Positive
3,RT @SortedEagle: According to news agency ANI'...,🇮🇳 દિવ્યમશુ સરકાર 🇮🇳,305340,5623,India,False,0,11,2020-11-22 08:05:55,RT According to news agency ANI s report Prime...,Neutral
4,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Ritesh Salot,121,12,,False,0,4,2020-11-22 08:05:55,RT Spoke to MulayamSinghYadav and greeted him ...,Positive


In [28]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Piyush cahturvedi,6842,102,India,False,0,4,2020-11-22 08:07:44,RT Spoke to MulayamSinghYadav and greeted him ...,Positive
1,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Надежда Румянцева,151647,1372,,False,0,4,2020-11-22 08:07:14,RT Spoke to MulayamSinghYadav and greeted him ...,Positive
2,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Shailesh,5637,55,Bharat,False,0,4,2020-11-22 08:06:38,RT Spoke to MulayamSinghYadav and greeted him ...,Positive
3,RT @SortedEagle: According to news agency ANI'...,🇮🇳 દિવ્યમશુ સરકાર 🇮🇳,305340,5623,India,False,0,11,2020-11-22 08:05:55,RT According to news agency ANI s report Prime...,Neutral
4,RT @timesofindia: Spoke to #MulayamSinghYadav ...,Ritesh Salot,121,12,,False,0,4,2020-11-22 08:05:55,RT Spoke to MulayamSinghYadav and greeted him ...,Positive


In [29]:
df[df['User_verified'] == True].count()

Tweets                 30
User                   30
User_statuses_count    30
user_followers         30
User_location          30
User_verified          30
fav_count              30
rt_count               30
tweet_date             30
clean_tweet            30
Sentiment              30
dtype: int64

In [30]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @timesofindia: Spoke to #MulayamSinghYadav and greeted him on his birthday. He is one of the senior-most and experienced leaders of our…

Clean tweet:
 RT Spoke to MulayamSinghYadav and greeted him on his birthday He is one of the senior most and experienced leaders of our

Sentiment of the tweet:
 Positive


In [31]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @ANI: Spoke to #MulayamSinghYadav and greeted him on his birthday. He is one of the senior-most and experienced leaders of our country w…

Clean tweet:
 RT Spoke to MulayamSinghYadav and greeted him on his birthday He is one of the senior most and experienced leaders of our country w

Sentiment of the tweet:
 Positive


In [32]:
n = 203
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @RavinderKapur2: ONE  LOOK  AT  NARENDRA  MODI AND  YOU  CAN  MAKE  OUT  HE  IS  A  CORRUPT  AND ARROGANT PRIME MINISTER .

Clean tweet:
 RT ONE LOOK AT NARENDRA MODI AND YOU CAN MAKE OUT HE IS A CORRUPT AND ARROGANT PRIME MINISTER

Sentiment of the tweet:
 Negative


In [33]:
df[df.Sentiment == 'Positive'].shape[0]

525

In [34]:
df[df.Sentiment == 'Neutral'].shape[0]

406

In [35]:
df[df.Sentiment == 'Negative'].shape[0]

69